## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [ ]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model

In [9]:
template = "{food}의 효과를 알려줘."

prompt = PromptTemplate.from_template(template)

model = ChatOpenAI()

chain = prompt | model

In [10]:
chain_reverse = model | prompt

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [13]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리", "food": "사과"}

In [14]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='사과는 다양한 영양소를 함유하고 있어 다양한 건강 효과를 가져다 줍니다. \n\n1. 항산화 작용: 사과에는 폴리페놀과 비타민 C가 풍부하게 함유되어 있어 항산화 작용을 하여 세포 손상을 예방하고 면역력을 강화시킵니다.\n\n2. 심혈관 건강: 사과에는 심혈관 질환을 예방하는데 도움을 주는 식이섬유와 칼륨이 풍부하게 함유되어 있습니다.\n\n3. 다이어트에 도움: 사과는 칼로리가 낮고 식이섬유가 풍부하여 살이 적게 찌는 데 도움을 줄 수 있습니다.\n\n4. 소화 촉진: 사과에는 소화를 원활하게 도와주는 식이섬유와 식이요소가 풍부하여 소화를 촉진시키는데 도움을 줍니다.\n\n5. 치아 건강: 사과를 먹음으로써 치아 주변을 마사지 하여 치아 주변을 깨끗하게 해주어 치아 건강에 도움을 줄 수 있습니다.\n\n이처럼 사과는 건강에 매우 좋은 과일이므로 꾸준히 섭취하는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 433, 'prompt_tokens': 22, 'total_tokens': 455, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2b3b1ecb-7610-4213-8f00-390363205368-0', usage_metadata={'input_tokens': 22,

In [15]:
chain_reverse.invoke(input)

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [30]:
chain = (PromptTemplate.from_template("오늘 점심으로 내가 {food}를 먹었어. 비슷한 취향이면서 색다른 음식을 먹고싶은데 5가지 추천해줘.")
    | ChatOpenAI()
    | PromptTemplate.from_template("{food}의 조리법을 알려줘.")
    | ChatOpenAI()
)

In [29]:
stream = chain.stream({"food": "김치찌개"})

In [31]:
stream_response(stream)

1. 돼지고기 김치볶음밥
- 재료: 밥, 돼지고기, 김치, 양파, 대파, 식용유, 간장, 설탕, 김치 국물
1) 먼저 팬에 식용유를 두르고 돼지고기를 볶아줍니다.
2) 돼지고기가 익으면 양파와 대파를 넣고 함께 볶아줍니다.
3) 김치를 넣고 볶은 후 김치 국물과 간장, 설탕을 넣어 양념해줍니다.
4) 밥을 넣고 골고루 볶은 후 완성!

2. 된장찌개
- 재료: 된장, 물, 무, 양파, 대파, 고추, 마늘, 돼지고기
1) 먼저 된장에 물을 넣고 잘 풀어줍니다.
2) 무, 양파, 대파, 고추, 마늘을 넣고 끓여줍니다.
3) 돼지고기를 넣고 익힌 후 된장을 넣어 간을 맞춰줍니다.
4) 끓인 후에 대파를 곁들여 완성!

3. 불고기
- 재료: 소고기, 양파, 당근, 대파, 간장, 설탕, 다진 마늘, 후추
1) 소고기를 양파, 당근, 대파와 함께 잘게 썰어줍니다.
2) 간장, 설탕, 다진 마늘, 후추를 넣고 재료를 버무려줍니다.
3) 뜨거운 팬에 볶아주면 완성!

4. 물냉면
- 물냉면은 물에 면을 삶아 식힌 후 물냉면 육수와 함께 내놓으면 됩니다.

5. 미역국
- 재료: 미역, 다시마, 물, 소금, 간장, 마늘, 대파
1) 미역과 다시마를 물에 불려 부드럽게 만들어줍니다.
2) 물에 간장, 마늘, 대파를 넣고 끓여 미역국을 만들어줍니다.

이렇게 각 요리의 조리법을 알려드렸습니다. 맛있게 즐기세요!

아래는 스트리밍을 출력하는 예시 입니다.

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 출력파서(Output Parser)


In [53]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [33]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = chain | output_parser

In [34]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"food": "삼겹살"}
chain.invoke(input)

'불고기의 경우, 먼저 돼지고기를 적당한 크기로 잘라준 뒤 간장, 설탕, 다진 마늘, 다진 양파, 참기름, 후추 등을 넣고 재워줍니다. 그 후 팬에 기름을 두르고 양념한 돼지고기를 넣어 볶아줍니다. 살짝 익은 후 불고기 소스를 넣고 약불에서 조리하여 완성합니다.\n\n소갈비찜은 먼저 소갈비를 적당한 크기로 썰어놓고 간장, 설탕, 다진 마늘, 다진 양파, 후추 등을 넣고 재워줍니다. 냄비에 갈비와 양념 재료를 넣고 살짝 끓여준 뒤 낮은 불에서 찜해줍니다. 부드럽게 익은 소갈비찜이 완성됩니다.\n\n닭갈비는 닭고기를 썰어 먹기 좋은 크기로 잘라준 뒤 간장, 고추장, 고추가루, 설탕, 다진 마늘, 다진 양파 등을 넣고 재워줍니다. 팬에 기름을 두르고 양념한 닭고기를 볶아줍니다. 야채와 함께 볶아 매콤하게 조리하여 완성합니다.\n\n장어구이는 미리 소금물에 장어를 밀가루에 묻혀 구워내어 고소한 특별한 맛을 느낄 수 있습니다.\n\n해산물찜은 다양한 해산물을 준비한 뒤 간장, 고추장, 다진 마늘, 다진 양파, 후추 등을 넣고 끓여줍니다. 해산물이 익으면 통깨를 뿌려 완성합니다.'

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [41]:
query_template = """
당신은 수준급 데이터 모델링 실력을 갖춘 음식 도메인 전문가이자 개발자입니다. 주어진 상황을 만족할 수 있는 RDB 테이블과 데이터 생성 SQL을 생성해주세요.
양식은 [FORMAT]을 참고해 작성해 주세요.

# 상황:
{condition}

# FORMAT:
- 테이블 생성 DDL SQL:
- 테이블별 데이터 생성 DML SQL: 
"""

In [48]:
sql_chain = (
    PromptTemplate.from_template(query_template) | 
    ChatOpenAI()
    # StrOutputParser
)

In [43]:
condition = f"""
- 다음 음식의 정보를 담아야 한다.
- 음식의 레시피 정보를 담아야 한다.
- 레시피는 단계별로 순서에 맞게 조회가 가능해야 한다.
"""
format = """

"""
sql_input = {
    "condition": condition,
}

In [44]:
sql_input

{'condition': '\n- 다음 음식의 정보를 담아야 한다.\n- 음식의 레시피 정보를 담아야 한다.\n- 레시피는 단계별로 순서에 맞게 조회가 가능해야 한다.\n'}

In [50]:
sql_output = sql_chain.invoke(sql_input)

In [57]:
print(StrOutputParser().invoke(sql_output))

# 테이블 생성 DDL SQL:

CREATE TABLE foods (
    food_id INT PRIMARY KEY,
    food_name VARCHAR(50) NOT NULL,
    food_type VARCHAR(50) NOT NULL
);

CREATE TABLE recipes (
    recipe_id INT PRIMARY KEY,
    food_id INT,
    recipe_name VARCHAR(100) NOT NULL,
    recipe_description TEXT,
    FOREIGN KEY (food_id) REFERENCES foods(food_id)
);

CREATE TABLE recipe_steps (
    step_id INT PRIMARY KEY,
    recipe_id INT,
    step_number INT,
    step_description TEXT,
    FOREIGN KEY (recipe_id) REFERENCES recipes(recipe_id)
);

# 테이블별 데이터 생성 DML SQL:

INSERT INTO foods (food_id, food_name, food_type) VALUES (1, 'Pasta', 'Italian');
INSERT INTO foods (food_id, food_name, food_type) VALUES (2, 'Sushi', 'Japanese');

INSERT INTO recipes (recipe_id, food_id, recipe_name, recipe_description) VALUES (1, 1, 'Spaghetti Aglio e Olio', 'Simple and delicious pasta dish with garlic and olive oil.');
INSERT INTO recipes (recipe_id, food_id, recipe_name, recipe_description) VALUES (2, 2, 'California Roll', '

In [58]:
type(sql_chain)

langchain_core.runnables.base.RunnableSequence

In [61]:
type(ChatOpenAI())

langchain_openai.chat_models.base.ChatOpenAI

In [62]:
prompt_template = PromptTemplate.from_template("{food}의 효능에 대해 알려줘")
model = ChatOpenAI()
parser = StrOutputParser()

In [63]:
type(prompt_template | parser)

langchain_core.runnables.base.RunnableSequence

In [64]:
type(model | parser)

langchain_core.runnables.base.RunnableSequence

In [65]:
type(parser | model)

langchain_core.runnables.base.RunnableSequence

In [66]:
type(parser | parser)

langchain_core.runnables.base.RunnableSequence

In [67]:
type(parser | None)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'NoneType'>

In [59]:
type(PromptTemplate.from_template("{food}의 효능에 대해 알려줘"))

langchain_core.prompts.prompt.PromptTemplate

In [60]:
type(PromptTemplate.from_template("{food}의 효능에 대해 알려줘") | ChatOpenAI())

langchain_core.runnables.base.RunnableSequence

In [45]:
sql_stream = sql_chain.stream(sql_input)

In [46]:
stream_response(sql_stream)

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [ ]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4-turbo")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [ ]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)